In [1]:
!pip install tensorflow==2.4.0 import_ipynb==0.1.3

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=c106cd14620f9f07c8e7c54f77c407322fdea1eb8699edefeb124b216ce08dde
  Stored in directory: /home/jovyan/.cache/pip/wheels/06/7e/ad/1cb03e935234186825cefc7e2c8f3451b4f654b5bc72232a7b
Successfully built import-ipynb


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50V2, MobileNetV3Small
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import Model
import import_ipynb
from codait_utils import *

importing Jupyter notebook from codait_utils.ipynb


In [3]:
unzip('.','data.zip')

In [4]:
import os.path  
import glob  
folder = glob.glob("data/*")
num_classes=len(folder)

In [11]:
batch_size = 32
img_height = 400
img_width = 400
input_shape = (img_width,img_height)


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  'data',
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

train_ds = train_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))
val_ds = val_ds.map(lambda x, y: (x, tf.one_hot(y, depth=num_classes)))




Found 930 files belonging to 25 classes.
Using 744 files for training.
Found 930 files belonging to 25 classes.
Using 186 files for validation.


In [15]:
def my_net(model,num_classes, freeze_layers=10,full_freeze='N'):
    x = model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    out = Dense(num_classes,activation='softmax')(x)
    model_final = Model(model.input,out)
    if full_freeze != 'N':
        for layer in model.layers[0:freeze_layers]:
            layer.trainable = False
    return model_final

In [16]:
#model = ResNet50V2(weights='imagenet',include_top=False)
#model = my_net(model)

In [17]:
model = tf.keras.applications.MobileNetV2(
    input_shape=(400,400,3), alpha=1.0, include_top=False,
    input_tensor=None, pooling=None, classes=2,
    classifier_activation='softmax'
)
model = my_net(model, num_classes=num_classes)

In [18]:
#model = tf.keras.applications.VGG16(
#    include_top=True, weights=None, input_tensor=None,
#    input_shape=(244, 244, 3), pooling=None, classes=2,
#    classifier_activation='softmax'
#)
#model = my_net(model)

In [19]:
model.compile(
    optimizer="adam",
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [20]:
model.fit(
    train_ds,
    batch_size=batch_size,
    epochs=100,
    validation_data=val_ds
)

Epoch 1/2
24/24 [==============================] - 219s 9s/step - loss: 2.0979 - accuracy: 0.5084 - val_loss: 7.5332 - val_accuracy: 0.5860
Epoch 2/2
24/24 [==============================] - 222s 9s/step - loss: 1.4670 - accuracy: 0.6376 - val_loss: 7.0216 - val_accuracy: 0.5538


In [21]:
model.save('model')


INFO:tensorflow:Assets written to: model/assets


In [22]:
zipdir('model.zip', 'model')

adding model to model.zip
